# Model Evaluation and Prediction
This notebook contains the evaluation and prediction steps for the time series models predicting world population.

In [18]:
# Import necessary libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from prophet import Prophet
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [3]:
# Load the dataset
file_path = '../data/processed/data_fe.csv'
data = pd.read_csv(file_path,parse_dates=['Year'],index_col='Year')
data.head()

,AG.LND.AGRI.ZS,EG.CFT.ACCS.ZS,EG.ELC.ACCS.ZS,EN.POP.DNST,ER.H2O.INTR.PC,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD,SE.ADT.LITR.ZS,SE.PRM.ENRL,SE.PRM.ENRL.TC.ZS,...,SP.POP.GROW,SP.POP.TOTL,SP.RUR.TOTL.ZS,SP.URB.TOTL.IN.ZS,GDP_per_Capita_lag1,Life_Expectancy_lag1,Crude_Birth_Rate_lag1,Pr_school_enrollment_lag1,Mortality_rate_lag1,Population_growth_rate_lag1
Year,,,,,,,,,,,,,,,,,,,,,
1961-01-01,35.879317,49.296068,73.351057,28.528475,13632.001963,1.439319e+12,468.456801,65.586548,401658848.0,28.105,...,1.350895,3.072470e+09,65.910435,34.089565,450.106029,50.894331,31.908511,90.8022,17.234125,1.350895
1962-01-01,35.952470,49.296068,73.351057,29.033819,13395.564612,1.542845e+12,493.411159,65.586548,401658848.0,28.105,...,1.771351,3.126894e+09,65.478880,34.521120,468.456801,52.846477,31.165497,90.8022,14.583294,1.350895
1963-01-01,36.035383,49.296068,73.351057,29.651986,13109.723539,1.664977e+12,521.369208,65.586548,401658848.0,28.105,...,2.129136,3.193470e+09,65.102013,34.897987,493.411159,55.208783,35.103391,90.8022,13.616499,1.771351
1964-01-01,36.117043,49.296068,73.351057,30.274183,12834.164741,1.827785e+12,560.587725,65.586548,401658848.0,28.105,...,2.098330,3.260480e+09,64.717882,35.282118,521.369208,55.542430,36.274663,90.8022,13.459129,2.129136
1965-01-01,36.213941,49.296068,73.351057,30.903380,12566.777472,1.990240e+12,597.985077,65.586548,401658848.0,28.105,...,2.078320,3.328243e+09,64.500479,35.499521,560.587725,56.034953,35.131852,90.8022,13.529275,2.098330


In [27]:
# Prepare the features and target variable
features =['GDP_per_Capita_lag1', 'Life_Expectancy_lag1', 'Crude_Birth_Rate_lag1',\
           'Pr_school_enrollment_lag1','Mortality_rate_lag1','Population_growth_rate_lag1']

#X = data[features]
X = data[features]
y = data['SP.POP.TOTL']

# Split the data into training and testing sets
split_year = '2010-01-01'
X_train = X[X.index < split_year]
X_test = X[X.index >= split_year]
y_train = y[y.index < split_year]
y_test = y[y.index >= split_year]

# Display the shapes of the training and testing sets
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((49, 6), (11, 6), (49,), (11,))

### ARIMA Model

In [17]:
# Train the ARIMA model
arima_model = sm.tsa.ARIMA(y_train, order=(5, 1, 0))
arima_model_fit = arima_model.fit()

# Make predictions
arima_predictions = arima_model_fit.forecast(steps=len(y_test))[0]

# Display the predictions
display(arima_predictions)


# Calculate MAE and MSE for ARIMA model
#arima_mae = mean_absolute_error(y_test, arima_predictions)
#arima_mse = mean_squared_error(y_test, arima_predictions)

#(arima_mae, arima_mse)

/opt/homebrew/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/opt/homebrew/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/opt/homebrew/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/opt/homebrew/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


7759897343.164727

In [26]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

sarima_data=data


# Split the data into training and test sets
train_data = sarima_data[sarima_data.index < '2020-01-01']
test_data = sarima_data[sarima_data.index >= '2020-01-01']

# Fit the SARIMA model
sarima_model = SARIMAX(train_data['SP.POP.TOTL'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarima_result = sarima_model.fit(disp=False)

# Make predictions
start_index = test_data.index[0]
end_index = test_data.index[-1]
sarima_predictions = sarima_result.predict(start=start_index, end=end_index, typ='levels')

# Combine actual and predicted values
sarima_results = test_data.copy()
sarima_results['predicted_population'] = sarima_predictions

# Calculate evaluation metrics
sarima_mae = mean_absolute_error(sarima_results['SP.POP.TOTL'], sarima_results['predicted_population'])
sarima_mse = mean_squared_error(sarima_results['SP.POP.TOTL'], sarima_results['predicted_population'])

# Display evaluation metrics
print(f'Mean Absolute Error (MAE): {sarima_mae}')
print(f'Mean Squared Error (MSE): {sarima_mse}')

print(sarima_results['predicted_population'])

Mean Absolute Error (MAE): 5697944.196465492
Mean Squared Error (MSE): 32466568066034.785
Year
2020-01-01    7.826970e+09
Name: predicted_population, dtype: float64


/opt/homebrew/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/opt/homebrew/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)
/opt/homebrew/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


### Prophet Model

In [28]:
# Prepare the data for Prophet
train_df = y_train.reset_index()
train_df.columns = ['ds', 'y']

# Train the Prophet model
prophet_model = Prophet()
prophet_model.fit(train_df)

# Make future dataframe for predictions
future = prophet_model.make_future_dataframe(periods=len(y_test), freq='AS')

# Make predictions
prophet_forecast = prophet_model.predict(future)
prophet_predictions = prophet_forecast['yhat'][-len(y_test):].values

# Calculate MAE and MSE for Prophet model
prophet_mae = mean_absolute_error(y_test, prophet_predictions)
prophet_mse = mean_squared_error(y_test, prophet_predictions)

(prophet_mae, prophet_mse)

17:34:34 - cmdstanpy - INFO - Chain [1] start processing
17:34:34 - cmdstanpy - INFO - Chain [1] done processing


(25428414.68296155, 798590186396194.5)

###  LSTM Model

In [29]:
# Prepare the data for LSTM
def create_dataset(X, y, time_step=1):
    Xs, ys = [], []
    for i in range(len(X)-time_step-1):
        Xs.append(X[i:(i+time_step), :])
        ys.append(y[i + time_step])
    return np.array(Xs), np.array(ys)

time_step = 3
X_train_lstm, y_train_lstm = create_dataset(X_train.values, y_train.values, time_step)
X_test_lstm, y_test_lstm = create_dataset(X_test.values, y_test.values, time_step)

# Define the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, return_sequences=True, input_shape=(time_step, X_train_lstm.shape[2])))
lstm_model.add(LSTM(50, return_sequences=False))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the LSTM model
lstm_model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=1, verbose=1)

# Make predictions
lstm_predictions = lstm_model.predict(X_test_lstm)

# Reshape the predictions to match the shape of y_test
lstm_predictions = lstm_predictions.flatten()

# Calculate MAE and MSE for LSTM model
lstm_mae = mean_absolute_error(y_test[-len(lstm_predictions):], lstm_predictions)
lstm_mse = mean_squared_error(y_test[-len(lstm_predictions):], lstm_predictions)

(lstm_mae, lstm_mse)

Epoch 1/50


/opt/homebrew/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 26314781112353685504.0000 
Epoch 2/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27553191845043372032.0000 
Epoch 3/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26761901913835307008.0000
Epoch 4/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26480330180101406720.0000 
Epoch 5/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 22619010270096261120.0000
Epoch 6/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 28186143305758670848.0000
Epoch 7/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26058130909174956032.0000
Epoch 8/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26403491909505908736.0000
Epoch 9/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 25344589644190187520.0000
Epoch 10/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26918753844607320064.0000
Epoch 11/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26769396185090228224.0000 
Epoch 12/50
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27219239377362223104.0

(7573542241.655135, 5.7386945386264945e+19)

### Select Best Model and Predict for 2020

In [30]:
# Based on the evaluation metrics, select the best model 

# Prepare the entire dataset for Prophet
all_data_df = y.reset_index()
all_data_df.columns = ['ds', 'y']

# Train the Prophet model on the entire dataset
final_prophet_model = Prophet()
final_prophet_model.fit(all_data_df)

# Make future dataframe for 2020 prediction
future_final = final_prophet_model.make_future_dataframe(periods=1, freq='AS')

# Make prediction for 2020
final_forecast = final_prophet_model.predict(future_final)
population_2020 = final_forecast['yhat'].values[-1]

population_2020

17:35:05 - cmdstanpy - INFO - Chain [1] start processing
17:35:05 - cmdstanpy - INFO - Chain [1] done processing


7916023533.334709

In [ ]:
7.821272e+09